# Julia nivel intermedio

Hemos visto varias cosas:

- asignación de variables, incluyendo diversos tipos numéricos(`Int64`, `Float64`, `BigFloat`, `Complex{Float64}`, `Rational{Int64}`, etc), cadenas, vectores, matrices

- ciclos (`for`...`end`; `while`...`end`) y rangos (`a:b`)

- diversas formas de condicionales

- funciones simples

Aquí seguiremos con esto, introduciendo ahora el concepto de funciones paramétricas, la forma de definir tipos arbitrarios y cómo escribir código que escribe o modifica código.

## Ejemplo: el método de Newton

Como bien sabemos, el método de Newton es un método iterativo para encontrar los ceros, o raíces, de una ecuación $f(x)=0$. A partir de una aproximación $x_0$, y denotando la derivada de $f(x)$ como $f'(x)$, tenemos:

\begin{equation}
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} .
\end{equation}



Para ejemplificar las cosas usaremos la función $f(x) = x^2-2$:

In [ ]:
f(x) = x^2 - 2.0

... y su derivada:

In [ ]:
f´(x) = 2*x

Entonces, el método de Newton vendría dado por el siguiente código:

In [ ]:
x_0 = 3.0

for i in 1:10  # Consideramos 10 iteraciones del método
    x_new = x_0 - f(x_0) / f´(x_0)
    println(i, "\t", x_new)
    x_0 = x_new
end
x_0

El error relativo es:

In [ ]:
(ans-sqrt(2.0))/sqrt(2.0)

Recordando la clase de cálculo I, el método de Newton converge si $x_0$ es *suficientemente* cercano a la raíz. Veamos qué pasa si *variamos* la condición inicial $x_0$.

In [ ]:
initial_guess = -3:0.125:3

In [ ]:
typeof(initial_guess)

In [ ]:
collect(initial_guess)   # use tab-completion <TAB> for long variable names!

Defino el arreglo de salida que, entrada por entrada, corresponderá a las condiciones iniciales definidas en `initial_conditions`.

In [ ]:
roots = zeros(length(initial_guess))

In [ ]:
roots = similar(initial_guess)  # `similar` copy la estructura de `initial_guess` (tipo y 
                                # tamaño en `roots`

El número de datos de `initial_guess` es precisamente 49:

In [ ]:
length(initial_guess)

Ahora, implementamos todo junto; abajo, `enumerate(initial_conditions)` regresa una *tupla* con un enetero, que es el índice de la condición inicial (`j`), y su valor (`x_0`)

In [ ]:
for (j, x_0) in enumerate(initial_guess)
    x = x_0

    # 100 iterates of Newton's method
    for i in 1:100
        x = x - f(x) / f´(x)
    end
    
    roots[j] = x
end

Veamos los resultados:

In [ ]:
showall(roots)

**Pregunta:** ¿Por qué hay un `NaN` entre las raíces obtenidas?

In [ ]:
roots[25]

Ahora vamos a visualizar esto. Para eso usaremos el paquete "PyPlot", que es una interface para usar "matplotlb".

Para instalarlo es necesario usar el siguiente comando:
```julia
    Pkg.add("PyPlot")
```

Otros paquetes *registrados* se pueden encontrar [aquí](http://pkg.julialang.org/); la lista empieza a ser larga!

Para empezar a usar un paquete ya instalado, en este caso "PyPlot", usamos el comando:

```julia
    using PyPlot
```

(Puede tardar un poco la primera vez que se utiliza.)

In [ ]:
using PyPlot

In [ ]:
figure(figsize=(6,4)) # este comando define el tamaño de la figura

plot(roots)           # este comando pinta las componentes "y" (`roots`); 
                      # en "x" se utiliza el índice del vector

In [ ]:
figure(figsize=(6,4)) # este comando define el tamaño de la figura

plot(initial_guess, roots)    # este comando pinta las componentes "y" (`roots`); 
                              # en "x" se utiliza `initial_guess`

title(L"Convergencia del método de Newton para $f(x)=x^2-2$", fontsize=10)

xlabel(L"$x_0$ (Condición inicial)")
ylabel(L"$x_{100}$ (Iterado de Newton final)")

## Rendimiento

La manera en que procedimos arriba **no** es la óptima para trabajar en Julia. Es **mucho** mejor poner código que será utilizado de manera repetida dentro de una función.

Entonces, lo anterior, lo empacamos dentro de una función.

In [ ]:
"""
    compute_roots()

This function computes the roots of `f(x)` (which must be defined before) 
for different initial iterates in the range [-20,20], and returns them 
packed in a vector.
"""
function compute_roots()
    initial_conditions = -20:0.125:20
    roots = similar(initial_conditions)

    for (j, x_0) in enumerate(initial_conditions)
        x = x_0

        # 100 iterations of Newton's method
        for i in 1:100
            x = x - f(x) / f´(x)
        end

        roots[j] = x
    end
    
    roots
end

Noten la salida de la definición de la función `compute_roots`:

```
    compute_roots (generic function with 1 method)
```

A esto volveremos más adelante...

In [ ]:
?compute_roots

**Ejercicio 1:** ¿Cuál es el tipo de `initial_conditions` y cual es el de `roots`?

In [ ]:
# ";" suprime la salida (el vector es demasiado largo)
roots = compute_roots();

¿Cuánto tiempo tardó esto?

El *macro* `@time` (que usamos la clase pasada) permite precisamente medir el tiempo de ejecución.

Julia *compila* cada función la primera vez que es utilizada. Es por eso que conviene compilar `@time`, y luego la ejecución de `@time` con la función que nos interesa medir. Así, para medir de manera consistente el tiempo de ejecución (modulo pequeñas fluctuaciones), es importante no incluir en esa medición la compilación de la función.

In [ ]:
@time 1;

In [ ]:
@time 1;

Las dos ejecuciones anteriores muestran que, al compilar (primer ejecución), julia utiliza espacio de memoria; una vez compiladas las cosas, el uso de memoria disminuye de manera importante. Incidentalmente, el uso exagerado de memoria muestra que las cosas pueden ser hechas de mejor manera...

In [ ]:
@time compute_roots();

In [ ]:
@time compute_roots();  # como `compute_roots` ya está compilada, no hay mucho cambio en el
                        # tiempo de ejecución o memoria usada

¿Y si queremos calcular los ceros de otra función?

- Habría que redefinir `f(x)` y `f'(x)`
- Recompilar `compute_roots()`

## Funciones genéricas

Lo que hemos hecho hasta ahora está bien, pero *no* es muy genérico, en el sentido de que no es muy cómodo la implementación si queremos considerar distintas funciones `f(x)`.

Para esto, *definimos* el siguiente *método* para la función `compute_roots`; noten que es el mismo nombre de la función que definimos antes.

In [ ]:
function compute_roots(f, f´)
    initial_conditions = -20:0.125:20
    roots = similar(initial_conditions)

    for (j, x_0) in enumerate(initial_conditions)
        x = x_0

        # 100 iterations of Newton's method
        for i in 1:100
            x = x - f(x) / f´(x)
        end

        roots[j] = x
    end
    
    roots
end

Es importante notar que, ahora, tenemos 2 métodos definidos para la *misma* función `compute_roots`. Esto es, **no** hemos sobreescrito la función `compute_roots`, sino que hemos creado un nuevo método. 

Los métodos se distinguen por el tipo de argumentos de la función: en un caso *no* hay ningún argumento, mientras que en el otro hay dos argumentos.

La instrucción `methods` ayuda a saber cuántos métodos hay asociados a una función, incluyendo información sobre el tipo de los parámetros; esto último es la "signatura de tipo" (*type signature*).

In [ ]:
methods(compute_roots)

**Ejercicio 2:** ¿De qué tipo son las variables f y f´?

Ejecutamos ahora la nueva implementación de las funciones:

In [ ]:
@time compute_roots(f, f´);

In [ ]:
@time compute_roots(f, f´);

Vale la pena notar que, la vieja implementación es más rápida que la nueva, incluyendo un mejor uso de la memoria. Esto se debe a que Julia v0.4 *no* sabe cómo inferir el resultado de $f$ y $f'$, cuando son pasadas como argumento de una función; esto cambia y se vuelve más eficiente en Julia v0.5. 

Como vimos anteriormente, la manera de notar que `compute_roots(f,f´)` va a dar problemas se obtiene usando el macro `@code_warntype`. 

En las siguientes dos ejecuciones, vale la pena *notar* en el apartado de variables que la variable `x` en un caso es un `Float64` y en el otro es `Any`, respectivamente para `compute_roots()` y `compute_roots(f,f´)`; obtener `Any` es *mala señal*:

In [ ]:
@code_warntype(compute_roots())

In [ ]:
@code_warntype(compute_roots(f, f´))

Otra manera de hacer esto mismo, pero usando funciones anónimas es la siguiente:

In [ ]:
@time compute_roots(x->x^2-2, x->2x);

In [ ]:
@time compute_roots(x->x^2-2, x->2x);

¿Cómo le podemos hacer si queremos guardar el tiempo de ejecución?

In [ ]:
@elapsed compute_roots(x->x^2-2, x->2x)

En este momento (versión 0.4 de Julia), las funciones anónimas son aún lentas, pero se pueden hacer trucos para que funcionen mejor. Esto tiene que ver con el hecho de que Julia no sabe el tipo de resultado de las funciones anónimas, en el momento de compilar la función.

Hagamos otro ejemplo un poco más interesante:

In [ ]:
@time roots = compute_roots( x -> (x-1)*(x-2)*(x-3), 
                             x->(x-2)*(x-3)+(x-1)*(x-3)+(x-1)*(x-2) );

In [ ]:
@time roots = compute_roots( x -> (x-1)*(x-2)*(x-3), 
                             x->(x-2)*(x-3)+(x-1)*(x-3)+(x-1)*(x-2) );

In [ ]:
figure(figsize=(6,4))
plot(-20:0.125:20, roots)
ylim(0,4)

In [ ]:
figure(figsize=(6,4))
plot(-20:0.125:20, roots, "g.-")
ylim(0,4)
xlim(1,3)

**Ejercicio 3:** Modificar y documentar (!) la función `compute_roots` de tal manera que la condición inicial sea un rango arbitrario que el usuario da a la entrada.

Usando esta función, ¿cómo se ve las raíces a las que converge la iteración del método de Newton para $f(x) = (x-1)(x-2)(x-3)$? En este caso vale la pena concentrarse en valores $x_0\in [1,3]$, usar muchos puntos y quizás hacer observar localmente la estructura.

In [ ]:
function compute_roots(f, f´, initial_conditions)
#     initial_conditions = -20:0.125:20
    roots = similar(initial_conditions)

    for (j, x_0) in enumerate(initial_conditions)
        x = x_0

        # 100 iterations of Newton's method
        for i in 1:100
            x = x - f(x) / f´(x)
        end

        roots[j] = x
    end
    
    roots
end

In [ ]:
figure(figsize=(6,4))
myrange = 1.0:1/8:3.0

plot(myrange, 
     compute_roots( x -> (x-1)*(x-2)*(x-3), 
            x->(x-2)*(x-3)+(x-1)*(x-3)+(x-1)*(x-2), 
            myrange ), 
    "g.-")


ylim(0,4)
xlim(1,3)

In [ ]:
figure(figsize=(6,4))
myrange = 1.3:1/364:1.7

plot(myrange, 
     compute_roots( x -> (x-1)*(x-2)*(x-3), 
            x->(x-2)*(x-3)+(x-1)*(x-3)+(x-1)*(x-2), 
            myrange ), 
    "g.-")


ylim(0,4)


In [ ]:
figure(figsize=(6,4))
myrange = 1.5:1/2^12:1.6

plot(myrange, 
     compute_roots( x -> (x-1)*(x-2)*(x-3), 
            x->(x-2)*(x-3)+(x-1)*(x-3)+(x-1)*(x-2), 
            myrange ), 
    "g.-")


ylim(0,4)


In [ ]:
figure(figsize=(6,4))
myrange = 1.552:1/2^16:1.5552

plot(myrange, 
     compute_roots( x -> (x-1)*(x-2)*(x-3), 
            x->(x-2)*(x-3)+(x-1)*(x-3)+(x-1)*(x-2), 
            myrange ), 
    "g.-")


ylim(0,4)


## Global scope

Las variables (asignaciones) que uno define en el Jupyter notebook o en el REPL, por ejemplo, ejecutando
```julia
    x = 3
```
están definidas en lo que se llama el *global scope*. Las que uno define internamente en ciclos (por ejemplo, `for... x = 4 ... end`) o en funciones, están definidas en el *local scope*, del ciclo o de la función.

Una cuestión **importante** es que Julia *penaliza* las variables definidas en el global scope; la razón es que dichas variables pueden eventualmente cambiar de tipo, por lo que deben ser guardadas como apuntadores.

(Si las variables cambian de tipo en el local scope, ya vimos que esto está muy penalizado.)


Si uno quiere usar, de manera eficiente, variables en el *global scope*, uno debe definirlas como *constantes*:
```julia
    const x = 3
```

La variable `x` será definida como de tipo `Int` en este ejemplo. Uno puede cambiarle el valor, pero no el tipo.

In [ ]:
const var_globalscope = 3

In [ ]:
var_globalscope

In [ ]:
isconst(var_globalscope)

In [ ]:
isconst(:var_globalscope)

In [ ]:
var_globalscope = 0

In [ ]:
var_globalscope

In [ ]:
isconst(:var_globalscope)

In [ ]:
var_globalscope = 1.125

In [ ]:
const var_globalscope = 1.125

## El método de Newton sobre los complejos

Ahora, implementaremos el método de Newton, para alguna función modelo ($f(z) = z^3-1$), pero usaremos condiciones iniciales en los complejos.

Las condiciones iniciales, igual que antes, las definiremos a partir de un `FloatRange{Float64}`, que usaremos tanto para la parte real como para la parte imaginaria de $z_0$.

A priori podríamos proceder como antes. Sin embargo, hay *sutilezas*, ya que la salida `roots` no será un vector, sino una matriz.

In [ ]:
const cc = complex(1.0,0.0)

f(z) = z^3 - cc
f´(z) = 3*z^2

**NOTA**: La siguiente función necesita las funciones $f$ y $f'$, lo que permite usarlas en contextos más generales. Esto, como vimos antes, tiene una penalización en la ejecución.

In [ ]:
function compute_complex_roots(f, f´, range=-5.0:0.125:5.0)  # default value

    L = length(range)
    
    ## Se define la matriz con todos los elementos inicialmente en cero
    roots = zeros(Complex128, L, L)  # set a matrix of appropriate length to zero

#     for (j, y) in enumerate(range)
#         for (i, x) in enumerate(range)
    for (j, y) in enumerate(range), (i, x) in enumerate(range)
            
            z = x + y*im
            
            for k in 1:1000
                z = z - f(z) / f´(z)
            end
            
            roots[i,j] = z
            
    end
    
    roots
end

In [ ]:
@time compute_complex_roots(f, f´, -5.0:1.0:5.0);

In [ ]:
@time croots = compute_complex_roots(f, f´, -5.0:1/32:5.0);

**Ejercicio 4:** Encuentren la implementación más rápida de `compute_complex_roots` considerando las dos opciones relacionadas con el ´rden en que aparecen los índices (i,j)en el doble ciclo.

¿Por qué `compute_complex_roots` tiene 2 métodos definidos?

Visualicemos los resultados: para esto usaremos `imshow`, que sirve para visualizar una matriz, y el código de colores lo definiremos a partir de la parte imaginaria de `roots`.

In [ ]:
imshow(imag(croots))

**NOTA** La librería `PyPlot` está basada en `matplotlib`; en la red hay muchos recursos para usar `matplotlib` y generar diversos tipos de gráficas.

## Definición de nuevos tipos o estructuras

Ahora veremos cómo crear nuevos tipos. Esto es algo impresionantemente útil, ya que permite definir un arreglo compacto de datos de interés. Dado que Julia se basa en el tipo, esto se traduce en velocidad.

A manera ilustrativa definiremos un nuevo tipo, que llamaremos `AuDiff`.

### Los campos de un `type`

Para entender un poco más lo que significa definir un nuevo tipo, consideremos los números complejos:

In [ ]:
z0 = complex(3,2) # z=3+2im

In [ ]:
typeof(ans)

La instrucción `fieldnames` da los campos internos de un tipo; vale la pena notar que el resultado consiste en un vector de *símbolos*. Estos símbolos son los nombres de los campos internos del tipo.

In [ ]:
fieldnames(z0)

In [ ]:
z0.re, z0.im

In [ ]:
typeof(ans)

In [ ]:
BigInt(1)//2

In [ ]:
typeof(ans)

In [ ]:
fieldnames(BigInt(1)//2)

### Definición del tipo `AuDiff`

La sintáxis básica para definir nuevos tipos es:

In [ ]:
type AuDiff
    fun :: Float64
    der :: Float64
end

En la celda anterior, el doble dos-puntos `::` sirve para *anotar* el tipo de las variables internas. Esto es, `AuDiff` constará de dos componentes, `fun` y `der`, ambas del tipo `Float64`.

Por **convención**, las estructuras de tipo empiezan con una *mayúscula*; las funciones empiezan en *minúsculas*.

Veamos cuántos métodos hemos definido en torno a `AuDiff`:

In [ ]:
methods(AuDiff)

Usando el resultado de `methods`, probamos con la función `call`:

In [ ]:
call(AuDiff, 1.0, 2.0)

In [ ]:
call(AuDiff, 1, 2)

In [ ]:
call(AuDiff, (1, 2)...) # "..." es el 'splat' operator

Los puntos suspensivos `...` implican que la *tupla* (1,2) será separada en sus componentes individuales, cada una considerada como un parámetro independiente.

**NOTA:** La función `call` de hecho es la forma genérica de crear cualquier tipo en Julia v0.4, pero se tornará obsoleta, por lo que no la usaremos. En cambio, usaremos la forma más explícita:

In [ ]:
x = AuDiff(1, 4)

In [ ]:
y = AuDiff(5, 6)

In [ ]:
fieldnames(AuDiff)

In [ ]:
x.fun

Vale la pena notar que uno puede modificar los campos internos de un tipo:

In [ ]:
x.fun = 3

In [ ]:
x

**NOTA:** Uno también puede definir estructuras en que **no** sea posible modificar los valores internos; un ejemplo es `Rational`.

Sin importar mucho el significado de cada componente, la pregunta es si podemos sumar dos de ellos. Problemos:

In [ ]:
x + y

Julia nos arroja un error ya que *no* está definida la función `+` para los argumentos de tipo `AuDiff`; el mensaje de la primer línea es perfectamente claro.

Entonces, dado que *no* está definido, y nos interesa usarlo, podemos simplemente definirlo. Por ejemplo, usando la siguiente definición:

In [ ]:
+(a::AuDiff, b::AuDiff) = AuDiff( a.fun+b.fun, a.der+b.der )

In [ ]:
x + y

De igual manera procedemos con las demás operaciones. Para evitar el mensaje, importaremos primero las funciones desde `Base` y luego las definiremos.

In [ ]:
import Base.- # "import + from Base"

-(a::AuDiff, b::AuDiff) = AuDiff(a.fun-b.fun, a.der-b.der)

In [ ]:
x - y

In [ ]:
x * y

In [ ]:
import Base: *, /, ^

*(a::AuDiff, b::AuDiff) = AuDiff( a.fun*b.fun, a.fun*b.der+a.der*b.fun )

function /(a::AuDiff, b::AuDiff)
    nn = a.fun/b.fun
    dd = (a.der - nn*b.der)/b.fun
    return AuDiff(nn, dd)
end

^(a::AuDiff, n::Int) = AuDiff(a.fun^n, n * a.der * a.fun^(n-1))

Veamos algunos ejemplos:

In [ ]:
x + y

In [ ]:
x - y

In [ ]:
x * y

In [ ]:
x / y

Y, si tengo funciones, esto jala de maravilla:

In [ ]:
g(x,y) = x^2-y^3

In [ ]:
g(x,y)

### Tipos parametrizados

El tipo `AuDiff` que definimos arriba funciona, pero es rígido en el sentido de que sus campos son *siempre* `Float64`. Por ejemplo:

In [ ]:
AuDiff(1//2, 1//1) + AuDiff(1//3, 1//1)

In [ ]:
AuDiff(complex(1,2), complex(1,1))

Hay situaciones en que nos interesa tener un tipo y que éste funcione para distintos tipos de signatura de los parámetros internos, por ejemplo, con `Complex` o `Rational` u otras cosas del estilo. Por ejemplo, cosas tipo:

In [ ]:
typeof(complex(1.0,2.0))

In [ ]:
typeof(complex(1,2))

In [ ]:
typeof( [1, 2.1] )

Noten arriba que el entero "1" es *promovido* a Float64 "1.0", que es el tipo del segundo elemento.

Trataremos de hacer esto; sin embargo, Julia **no** permite redefinir las estructuras. Por esto, borraremos **todos** los nombres que tenemos definidos, e iniciaremos de nuevo.

In [ ]:
workspace()

In [ ]:
x

In [ ]:
type AuDiff{T}
    fun :: T
    der :: T
end

En la definición de `AuDiff`, `T` representa el tipo de la estructura que creamos y *parametriza* a `AuDiff` lo que se indica con `{T}`. Dada esta definición, ambas componentes `x` y `y` tienen que ser del (mismo) tipo `T`.

In [ ]:
x = AuDiff(1,3)

y = AuDiff(2.0,3.5)

@show(x, y)

El definir una estructura parametrizada hace que, por default, si los parámetros **no** cumplen la signatura del tipo, haya un error:

In [ ]:
AuDiff(1, 2.3)

Noten que la actual definición de `AuDiff{T}` (parametrizada) es *demasiado* flexible, esto es, permite definir ciertos tipos que, quizás, no nos interesa considerar.

In [ ]:
AuDiff("Soy", "Luis")

Las funciones, de hecho, también pueden ser parametrizadas:

In [ ]:
import Base.+

+{T}(a::AuDiff{T}, b::AuDiff{T}) = AuDiff{T}(a.fun+b.fun, a.der+b.der)

Noten arriba que la función, de hecho, está parametrizada, y que el parámetro está relacionado con el parámetros del tipo de los argumentos. 



In [ ]:
AuDiff(1,3) + AuDiff(-1,2)

In [ ]:
AuDiff(1,3) + AuDiff(-1//2,2//2)

El error es claro: no hemos definido cómo sumar `AuDiff{Int64}` con `AuDiff{Rational{Int64}}`; únicamente hemos definido la suma de dos `AuDiff{Int64}` o de dos `AuDiff{Rational{Int64}}`, pero ambos deben ser del mismo tipo, incluyendo sus parametrizaciones.

Uno, de hecho, podría haber definido la misma operación sin incluir la parte paramétrica; esto tiene consecuencias interesantes, que de hecho, a veces son deseables. Así, tenemos:

In [ ]:
import Base.-
-(a::AuDiff, b::AuDiff) = AuDiff(a.fun-b.fun, a.der-b.der)

In [ ]:
AuDiff(1,3) - AuDiff(-1,2)

In [ ]:
AuDiff(1,3) - AuDiff(3.0,1.1)

In [ ]:
AuDiff(1,3) - AuDiff("Soy", "Luis")

Gracias a que *no* impusimos que `a` y `b` sean *ambos* `AuDiff{T}`, julia puede restarlos.

### Promotion and convertion

La pregunta es, cómo conseguir que *no* nos de un error la instrucción `Vector2D(1, 3.2)`. La respuesta tiene que ver con definir reglas de conversión y promoción. (En este caso concreto, como veremos abajo, una regla de promoción es suficiente.)

Convertir entre tipos (donde esto tenga sentido) se logra usando `convert`:

In [ ]:
convert(Float64, 1//2)

In [ ]:
convert(Array{Float64,1}, [1//2, 1//3])

In [ ]:
x = convert(Rational{Int128}, 0.5)

In [ ]:
typeof(x)

La operación de conversión debe tener sentido; por ejemplo, ¿qué esperamos que nos de `convert(Int64, 0.6)`?

In [ ]:
convert(Int64, 0.6)

También, existe la promoción de varios objetos a un tipo común:

In [ ]:
tup = promote(1, 1//2, BigInt(2))

In [ ]:
typeof(tup)

Como vimos, Julia define un árbol jerárquico en la organización de los tipos. Para entender esto veamos dos casos:

In [ ]:
? Int64

In [ ]:
? Rational

In [ ]:
? Signed

Esto muestra que hay ciertos tipos *concretos* (como `Int64`, `Float64`) y *otros* que son abstractos; los segundos en algún sentido agrupan varios tipos distintos.

La instrucción `super` permite ver qué tipo está directamente arriba en la estructura del árbol:

In [ ]:
super(Float64)

In [ ]:
super(AbstractFloat)

In [ ]:
super(Real)

... y la instrucción subtypes, qué está por debajo:

In [ ]:
subtypes(Real)

In [ ]:
subtypes(Number)

La estructura de árbol tiene un tope *por arriba*, `Any`:

In [ ]:
super(Number)

In [ ]:
super(Any)

In [ ]:
subtypes(Any)

Vale la pena notar que existe, entre varias cosas, el tipo `Void` (antes `Nothing`)

In [ ]:
? Void

Volviendo al punto de cómo hacer para que `Vector2D{1, 3.2}` funcione, podemos definir un nuevo *método*, para la creación del tipo `Vector2D`, que involucre una promoción de tipos, de la siguiente manera:

In [ ]:
AuDiff(a, b) = AuDiff(promote(a,b)...)

In [ ]:
methods(AuDiff)

In [ ]:
AuDiff(1, 3.2)

Obviamente, esto sólo tendrá sentido cuando la promoción tiene sentido; cuando no la tiene, arrojará un error:

In [ ]:
AuDiff("Hola", 3.2)

Por último, uno puede restringir el tipo de parámetros de una estructura, usando la notación ` T <: R` que significa que el parámetro `T` es subtipo de `R`. Un ejemplo es `T <: Real`.

In [ ]:
workspace()

In [ ]:
type AuDiff{T<:Real}
    fun :: T
    der :: T
end

Agregamos la promoción de los tipos:

In [ ]:
AuDiff(a, b) = AuDiff(promote(a,b)...)

In [ ]:
AuDiff(1, 3.2)

In [ ]:
AuDiff("Soy", "Luis")

La notación `T <: R` de hecho puede ser utilizada para saber si un tipo es subtipo de otro:

In [ ]:
Int64 <: Real

In [ ]:
Int64 <: Void

In [ ]:
Void <: Int64

Las dos últimas instrucciones muestran que `Void` e `Int64` pertenecen a dos ramas distintas del árbol jerárquico de tipos; en otras palabras, el tipo superior común entre `Int64` y `Void` es `Any`:

In [ ]:
promote_type(Int64, Void)

Finalmente, para definir que algo una estructura específica es un subtipo de algo específico, usamos `<:` nuevamente:

In [ ]:
workspace()

In [ ]:
type AuDiff{T<:Real} <: Real
    fun :: T
    der :: T
end

In [ ]:
AuDiff(a, b) = AuDiff(promote(a,b)...)

Agregamos otro constructor externo:

In [ ]:
AuDiff{T<:Real}(a::T) = AuDiff(a, zero(T))

In [ ]:
AuDiff(2.1)

Lo siguiente sirve para evitar muchas definiciones...

In [ ]:
import Base: convert

convert(::Type{AuDiff}, b::Real) = AuDiff(b)
convert{T<:Real, S<:Real}(::Type{AuDiff{T}}, b::S) = AuDiff(convert(T,b))
convert{T<:Real}(::Type{AuDiff{T}}, b::T) = AuDiff(b)
convert{T<:Real, S<:Real}(::Type{AuDiff{T}}, b::AuDiff{S}) = 
    AuDiff(convert(T,b.fun),convert(T,b.der))


In [ ]:
convert(AuDiff, 1.2)

In [ ]:
convert(AuDiff{Float64},1)

In [ ]:
import Base: promote_rule, promote

promote_rule{T<:Real}(::Type{AuDiff{T}}, ::Type{T}) = AuDiff{T}
promote_rule{T<:Real, S<:Real}(::Type{AuDiff{T}}, ::Type{S}) = AuDiff{promote_type(T, S)}

In [ ]:
promote_type(Float64,AuDiff{Float64})

Extendemos las operaciones aritméticas

In [ ]:
import Base: +, -, *, /, ^

+(a::AuDiff, b::AuDiff) = AuDiff(a.fun+b.fun, a.der+b.der)

-(a::AuDiff, b::AuDiff) = AuDiff(a.fun-b.fun, a.der-b.der)

*(a::AuDiff, b::AuDiff) = AuDiff(a.fun*b.fun, a.fun*b.der+a.der*b.fun)

function /(a::AuDiff, b::AuDiff)
    nn = a.fun/b.fun
    dd = (a.der - nn*b.der)/b.fun
    return AuDiff(nn, dd)
end

^(a::AuDiff, n::Int) = AuDiff(a.fun^n, n * a.der * a.fun^(n-1))

In [ ]:
AuDiff(2,1) + 2

**Ejercicio 5:** ¿Qué representa cada parte de la estructura `AuDiff` que definimos? ¿Qué se puede calcular usándola? 

Para guiar la respuesta, consideren un polinomio sencillo, y jugando con las operaciones definidas para `x::AuDiff{T}`, en particular para `x = AuDiff(1//2,1//1)` aclaren qué representan los campos de `x` y de la respuesta obtenida al evaluar `p(x)`.

Escriban clara y detalladamente, y añadan todos los métodos nuevos que sean necesarios.